In [1]:
# install librariry
!pip install ultralytics

In [3]:
# import libraries
import numpy as np
import pandas as pd
import cv2
import torch

In [7]:
# load yolo model for player detection
from ultralytics import YOLO
model = YOLO("best.pt")

In [9]:
model.info()

YOLOv5x summary: 285 layers, 97,203,260 parameters, 0 gradients, 246.9 GFLOPs


(285, 97203260, 0, 246.91051520000002)

In [13]:
# Load pretrained ResNet50 model for visual feature extraction
from torchvision import models, transforms
resnet = models.resnet50(pretrained=True)
resnet.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet.to(device)

# Image transformation for ResNet
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Function to extract embedding from player crop using ResNet
def extract_embedding(image):
    with torch.no_grad():
        tensor = transform(image).unsqueeze(0).to(device)
        embedding = resnet(tensor)
    return embedding.flatten().cpu().numpy()

In [15]:
# Function to process a video and extract player embeddings
def process_video(video_path, label, max_frames=100):
    cap = cv2.VideoCapture(video_path)
    player_features = {}
    id_counter = 0
    frame_count = 0

    while cap.isOpened() and frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)

        for result in results:
            boxes = result.boxes
            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf)
                if conf < 0.5:
                    continue  # Skip low-confidence detections

                crop = frame[y1:y2, x1:x2]
                if crop.size == 0:
                    continue

                try:
                    emb = extract_embedding(crop)
                    player_features[id_counter] = emb
                    id_counter += 1
                except Exception as e:
                    print(f"[{label}] Skipped player due to error: {e}")
                    continue

        frame_count += 1

    cap.release()
    print(f"[{label}] Processed {id_counter} players from {frame_count} frames")
    return player_features

In [17]:
# Extract features from both videos
broadcast_features = process_video("broadcast.mp4", "broadcast")
tacticam_features = process_video("tacticam.mp4", "tacticam")


0: 384x640 3 players, 1237.0ms
Speed: 13.2ms preprocess, 1237.0ms inference, 12.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 players, 1347.9ms
Speed: 20.3ms preprocess, 1347.9ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 players, 1285.8ms
Speed: 6.2ms preprocess, 1285.8ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 players, 1217.1ms
Speed: 6.5ms preprocess, 1217.1ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 players, 1343.8ms
Speed: 5.0ms preprocess, 1343.8ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 players, 1 referee, 1369.5ms
Speed: 5.5ms preprocess, 1369.5ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 goalkeeper, 12 players, 1 referee, 1127.4ms
Speed: 4.6ms preprocess, 1127.4ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 1 goalkeeper, 14 playe

In [23]:
# cosine similarity between feature vectors
from sklearn.metrics.pairwise import cosine_similarity
b_ids = list(broadcast_features.keys())
t_ids = list(tacticam_features.keys())

broadcast_vecs = [broadcast_features[i] for i in b_ids]
tacticam_vecs = [tacticam_features[i] for i in t_ids]

sim_matrix = cosine_similarity(tacticam_vecs, broadcast_vecs)

# Match tacticam players to broadcast players using max similarity
mapping = {}
for i, t_id in enumerate(t_ids):
    best_match_index = np.argmax(sim_matrix[i])
    b_id = b_ids[best_match_index]
    mapping[t_id] = b_id

print("\n Player ID Mapping (Tacticam ➝ Broadcast):")
for t_id, b_id in mapping.items():
    print(f"Tacticam ID {t_id} ➝ Broadcast ID {b_id}")


 Player ID Mapping (Tacticam ➝ Broadcast):
Tacticam ID 0 ➝ Broadcast ID 494
Tacticam ID 1 ➝ Broadcast ID 248
Tacticam ID 2 ➝ Broadcast ID 28
Tacticam ID 3 ➝ Broadcast ID 902
Tacticam ID 4 ➝ Broadcast ID 108
Tacticam ID 5 ➝ Broadcast ID 912
Tacticam ID 6 ➝ Broadcast ID 865
Tacticam ID 7 ➝ Broadcast ID 419
Tacticam ID 8 ➝ Broadcast ID 821
Tacticam ID 9 ➝ Broadcast ID 766
Tacticam ID 10 ➝ Broadcast ID 65
Tacticam ID 11 ➝ Broadcast ID 68
Tacticam ID 12 ➝ Broadcast ID 1144
Tacticam ID 13 ➝ Broadcast ID 1073
Tacticam ID 14 ➝ Broadcast ID 1101
Tacticam ID 15 ➝ Broadcast ID 902
Tacticam ID 16 ➝ Broadcast ID 66
Tacticam ID 17 ➝ Broadcast ID 93
Tacticam ID 18 ➝ Broadcast ID 675
Tacticam ID 19 ➝ Broadcast ID 1098
Tacticam ID 20 ➝ Broadcast ID 863
Tacticam ID 21 ➝ Broadcast ID 1098
Tacticam ID 22 ➝ Broadcast ID 349
Tacticam ID 23 ➝ Broadcast ID 902
Tacticam ID 24 ➝ Broadcast ID 832
Tacticam ID 25 ➝ Broadcast ID 865
Tacticam ID 26 ➝ Broadcast ID 66
Tacticam ID 27 ➝ Broadcast ID 863
Tacticam ID 28 

In [25]:
# Save the mapping to CSV
df = pd.DataFrame([{"tacticam_id": t, "broadcast_id": b} for t, b in mapping.items()])
df.to_csv("player_id_mapping.csv", index=False)
print("\n Mapping saved to 'player_id_mapping.csv'")


 Mapping saved to 'player_id_mapping.csv'
